# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-30-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-30-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-31 05:22:42,33.93911,67.709953,55008,2400,47609,4999.0,Afghanistan,141.305791,4.363002
1,NaN,NaN,NaN,Albania,2021-01-31 05:22:42,41.15330,20.168300,77251,1369,46927,28955.0,Albania,2684.376955,1.772145
2,NaN,NaN,NaN,Algeria,2021-01-31 05:22:42,28.03390,1.659600,107122,2888,73152,31082.0,Algeria,244.286094,2.695991
3,NaN,NaN,NaN,Andorra,2021-01-31 05:22:42,42.50630,1.521800,9885,101,9045,739.0,Andorra,12793.632304,1.021750
4,NaN,NaN,NaN,Angola,2021-01-31 05:22:42,-11.20270,17.873900,19782,464,18015,1303.0,Angola,60.189371,2.345567


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,54403,54483,54559,54595,54672,54750,54854,54891,54939,55008
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,69916,70655,71441,72274,72812,73691,74567,75454,76350,77251
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,104852,105124,105369,105596,105854,106097,106359,106610,106887,107122


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2363,2370,2373,2378,2385,2389,2389,2397,2399,2400
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1296,1303,1310,1315,1324,1332,1339,1350,1358,1369
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2853,2856,2861,2863,2866,2871,2877,2881,2884,2888


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,46887,46912,46943,47298,47365,47459,47549,47583,47606,47609
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,42426,42895,43384,43890,44291,44880,45383,45906,46420,46927
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,71343,71552,71755,71945,72143,72336,72538,72745,72956,73152


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5270,5327,5358,5376,5407,5440,5499,5554,5596,5596
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,16798,16981,17128,17256,17333,17496,17629,17779,17922,17922
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1778,1793,1805,1827,1834,1882,1898,1920,1931,1931


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,61,61,62,62,62,65,67,69,69,69
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,188,188,189,189,189,216,217,225,224,224
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,36,37,38,38,38,40,40,40,40,40


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
651,1001.0,Autauga,Alabama,US,2021-01-31 05:22:42,32.539527,-86.644082,5596,69,0,5527.0,"Autauga, Alabama, US",10016.288103,1.233024
688,1075.0,Lamar,Alabama,US,2021-01-31 05:22:42,33.779950,-88.096680,1235,27,0,1208.0,"Lamar, Alabama, US",8946.034046,2.186235
689,1077.0,Lauderdale,Alabama,US,2021-01-31 05:22:42,34.901719,-87.656247,8134,151,0,7983.0,"Lauderdale, Alabama, US",8771.797388,1.856405


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,26069046,439439,0
India,10746174,154274,10423109
Brazil,9176975,223945,8148671


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,26069046,439439,0,25629607,2021-01-31 05:22:42,37.936303,-91.379001
India,10746174,154274,10423109,168791,2021-01-31 05:22:42,23.088275,81.806127
Brazil,9176975,223945,8148671,804359,2021-01-31 05:22:42,-12.669522,-48.480493
United Kingdom,3806993,105777,9580,3691636,2021-01-31 05:22:42,30.744028,-35.367255
Russia,3790265,71556,3244856,473853,2021-01-31 05:22:42,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3308455,40678,0
Texas,2363266,36924,0
Florida,1713589,26360,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3308455,40678,0,3267777,2021-01-31 05:22:42,37.843962,-120.728594
Texas,2363266,36924,0,2326342,2021-01-31 05:22:42,31.660643,-98.653069
Florida,1713589,26360,0,1687229,2021-01-31 05:22:42,28.940755,-82.700744
New York,1408698,43453,0,1365245,2021-01-31 05:22:42,42.544151,-75.474183
Illinois,1120528,21146,0,1099382,2021-01-31 05:22:42,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1111089,16647,0
Arizona,Maricopa,469294,7442,0
Illinois,Cook,451581,9374,0
Florida,Miami-Dade,370642,4845,0
Texas,Harris,314697,4242,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1111089,16647,0,1094442,2021-01-31 05:22:42,34.308284,-118.228241,6037.0
Arizona,Maricopa,469294,7442,0,461852,2021-01-31 05:22:42,33.348359,-112.491815,4013.0
Illinois,Cook,451581,9374,0,442207,2021-01-31 05:22:42,41.841448,-87.816588,17031.0
Florida,Miami-Dade,370642,4845,0,365797,2021-01-31 05:22:42,25.611236,-80.551706,12086.0
Texas,Harris,314697,4242,0,310455,2021-01-31 05:22:42,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-28,54891,75454,106610,9779,19672,215,1905524,166669,28799,410230,...,3754448,39887,78556,1,125364,1651,157593,2120,50319,32646
2021-01-29,54939,76350,106887,9837,19723,218,1915362,166728,28806,411730,...,3783593,40529,78602,1,125776,1657,158168,2120,51624,32952
2021-01-30,55008,77251,107122,9885,19782,218,1922264,166901,28811,413208,...,3806993,41181,78711,1,126323,1767,158559,2120,53352,33273


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-28,2397,1350,2881,100,464,6,47601,3067,909,7607,...,103324,415,621,0,1171,35,1812,615,705,1160
2021-01-29,2399,1358,2884,100,464,6,47775,3069,909,7658,...,104572,425,621,0,1177,35,1823,615,728,1178
2021-01-30,2400,1369,2888,101,464,7,47931,3071,909,7703,...,105777,431,621,0,1183,35,1831,615,745,1193


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-28,47583,45906,72745,8942,17692,174,1693146,156645,26034,387787,...,9349,32306,77047,1,117447,1430,147620,1425,42771,24419
2021-01-29,47606,46420,72956,8986,17918,175,1703459,156910,26045,389476,...,9404,33146,77084,1,117867,1448,148140,1425,44133,24872
2021-01-30,47609,46927,73152,9045,18015,175,1712868,157314,26057,391076,...,9580,33914,77191,1,118385,1456,148583,1426,45443,25361


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5270,5327,5358,5376,5407,5440,5499,5554,5596,5596
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1196,1197,1199,1202,1206,1213,1220,1230,1235,1235
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,7733,7798,7847,7871,7946,7986,8049,8088,8134,8134
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2491,2507,2533,2541,2558,2569,2597,2616,2628,2628
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,12901,13036,13137,13205,13261,13378,13482,13602,13679,13679


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-28,452734,53877,0,743232,290856,3273905,391737,248765,76495,49,...,717583,2330272,342445,11523,2384,493674,307809,118562,587580,51430
2021-01-29,455582,54062,0,748260,292563,3293531,393459,250023,76935,49,...,722491,2348188,343962,11658,2398,497912,309801,119467,589384,51690
2021-01-30,455582,54216,0,753379,294387,3308455,395019,250023,77551,49,...,724742,2363266,345430,11831,2398,502221,311597,120340,590997,51704


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-28        5554   17779    1920  2271   5612    1079   1788   11833   
2021-01-29        5596   17922    1931  2284   5655    1075   1800   11918   
2021-01-30        5596   17922    1931  2284   5655    1075   1800   11918   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-28         3159     1682  ...    2525    549     2861      648   
2021-01-29         3174     1688  ...    2532    552     2868      649   
2021-01-30         3174     1688  ...    2527    553     2869      649   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-28           3440  3028  1957          0      864    610  
2021-01-29           3464  3078  1959          0      864    610  
2021-01-30           3472  3078  1960          0      864    610  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-28,7340,262,0,12819,4784,39574,5569,7020,1075,0,...,9417,36292,1620,172,24,6308,4243,1983,6334,596
2021-01-29,7566,262,0,13022,4831,40238,5594,7046,1078,0,...,9461,36650,1655,172,24,6379,4285,2006,6390,596
2021-01-30,7566,262,0,13098,4838,40678,5620,7046,1081,0,...,9574,36924,1663,173,24,6449,4285,2015,6430,596


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-28          69     225      40   51     98      29     60     231   
2021-01-29          69     224      40   51     98      29     60     232   
2021-01-30          69     224      40   51     98      29     60     232   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-28           76       35  ...      20     11       19        7   
2021-01-29           76       35  ...      20     11       19        7   
2021-01-30           76       35  ...      20     11       19        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-28             32     6    12          0       25      4  
2021-01-29             32     6    12          0       25      4  
2021-01-30             32     6    12          0       25      4  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-28,0.000675,0.011895,0.002360,0.006484,0.004699,0.000000,0.004995,0.001454,0.000174,0.003545,...,0.007733,0.014214,0.000586,0.0,0.003249,0.063104,0.003803,0.0,0.028787,0.010587
2021-01-29,0.000874,0.011875,0.002598,0.005931,0.002593,0.013953,0.005163,0.000354,0.000243,0.003656,...,0.007763,0.016095,0.000586,0.0,0.003286,0.003634,0.003649,0.0,0.025935,0.009373
2021-01-30,0.001256,0.011801,0.002199,0.004880,0.002991,0.000000,0.003603,0.001038,0.000174,0.003590,...,0.006185,0.016087,0.001387,0.0,0.004349,0.066385,0.002472,0.0,0.033473,0.009741


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-28,0.003349,0.008215,0.001390,0.020408,0.004329,0.000000,0.003500,0.003599,0.0,0.005685,...,0.012137,0.019656,0.0,NaN,0.005150,0.0,0.002212,0.0,0.024709,0.033868
2021-01-29,0.000834,0.005926,0.001041,0.000000,0.000000,0.000000,0.003655,0.000652,0.0,0.006704,...,0.012079,0.024096,0.0,NaN,0.005124,0.0,0.006071,0.0,0.032624,0.015517
2021-01-30,0.000417,0.008100,0.001387,0.010000,0.000000,0.166667,0.003265,0.000652,0.0,0.005876,...,0.011523,0.014118,0.0,NaN,0.005098,0.0,0.004388,0.0,0.023352,0.012733


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-28,0.000715,0.011524,0.002854,0.010281,0.005113,0.000000,0.006290,0.003794,0.000115,0.003717,...,0.011140,0.022406,0.000649,0.0,0.002835,0.000000,0.004669,0.000000,0.036069,0.030903
2021-01-29,0.000483,0.011197,0.002901,0.004921,0.012774,0.005747,0.006091,0.001692,0.000423,0.004355,...,0.005883,0.026001,0.000480,0.0,0.003576,0.012587,0.003523,0.000000,0.031844,0.018551
2021-01-30,0.000063,0.010922,0.002687,0.006566,0.005414,0.000000,0.005523,0.002575,0.000461,0.004108,...,0.018715,0.023170,0.001388,0.0,0.004395,0.005525,0.002990,0.000702,0.029683,0.019661


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-28,0.008123,0.003857,NaN,0.006324,0.006548,0.007154,0.003692,0.005765,0.008743,0.0,...,0.002483,0.009116,0.005169,0.012655,0.004635,0.010482,0.008254,0.006682,0.003381,0.001207
2021-01-29,0.006291,0.003434,NaN,0.006765,0.005869,0.005995,0.004396,0.005057,0.005752,0.0,...,0.006840,0.007688,0.004430,0.011716,0.005872,0.008585,0.006472,0.007633,0.003070,0.005055
2021-01-30,0.000000,0.002849,NaN,0.006841,0.006235,0.004531,0.003965,0.000000,0.008007,0.0,...,0.003116,0.006421,0.004268,0.014840,0.000000,0.008654,0.005797,0.007307,0.002737,0.000271


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-28      0.010002  0.008509  0.011591  0.004423  0.007902  0.004655   
2021-01-29      0.007562  0.008043  0.005729  0.005724  0.007662 -0.003707   
2021-01-30      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-28      0.012458  0.008781  0.007013  0.005380  ... -0.000396   
2021-01-29      0.006711  0.007183  0.004748  0.003567  ...  0.002772   
2021-01-30      0.000000  0.000000  0.000000  0.000000  ... -0.001975   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-28     -0.001818  0.002804  0.001546   0.003501  0.002649  0.004620   
2021-01-29      0.005464  0.002447  0.001543   0.006977  0.016513  0.001022   
2021-01-30      0.001812  0.000349  0.000000   0.002309  0.000000  0.000510   

Province_State                             
Admin2         Unassigned Washakie Weston  
2021-01-28            NaN      0.0    0.0  
2021-01-29            NaN      0.0    0.0  
2021-01-30            NaN      0.0    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-28,0.023424,0.003831,NaN,0.013921,0.008857,0.019502,0.003062,0.006307,0.002799,NaN,...,0.010842,0.011567,0.000000,0.000000,0.0,0.012845,0.007599,0.015361,0.005078,0.0
2021-01-29,0.030790,0.000000,NaN,0.015836,0.009824,0.016779,0.004489,0.003704,0.002791,NaN,...,0.004672,0.009864,0.021605,0.000000,0.0,0.011256,0.009899,0.011599,0.008841,0.0
2021-01-30,0.000000,0.000000,NaN,0.005836,0.001449,0.010935,0.004648,0.000000,0.002783,NaN,...,0.011944,0.007476,0.004834,0.005814,0.0,0.010974,0.000000,0.004487,0.006260,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2021-01-28      0.029851  0.036866     0.0  0.0  0.031579     0.0    0.0   
2021-01-29      0.000000 -0.004444     0.0  0.0  0.000000     0.0    0.0   
2021-01-30      0.000000  0.000000     0.0  0.0  0.000000     0.0    0.0   

Province_State                                ... Wyoming                  \
Admin2           Calhoun  Chambers  Cherokee  ...    Park Platte Sheridan   
2021-01-28      0.013158  0.013333  0.060606  ...     0.0    0.0      0.0   
2021-01-29      0.004329  0.000000  0.000000  ...     0.0    0.0      0.0   
2021-01-30      0.000000  0.000000  0.000000  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-28          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-01-29          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-01-30          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-28,0.001149,0.011586,0.002380,0.006523,0.003639,0.019015,0.005132,0.001211,0.000208,0.003602,...,0.007225,0.015409,0.000571,0.0,0.003360,0.032098,0.003640,0.000189,0.027712,0.010574
2021-01-29,0.001012,0.011731,0.002489,0.006227,0.003116,0.016484,0.005148,0.000782,0.000225,0.003629,...,0.007494,0.015752,0.000578,0.0,0.003323,0.017866,0.003644,0.000095,0.026823,0.009974
2021-01-30,0.001134,0.011766,0.002344,0.005553,0.003054,0.008242,0.004376,0.000910,0.000199,0.003609,...,0.006839,0.015920,0.000982,0.0,0.003836,0.042126,0.003058,0.000047,0.030148,0.009858


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-28,0.002202,0.006879,0.001571,0.013994,0.003146,0.000006,0.003751,0.002610,1.282107e-13,0.006172,...,0.013468,0.020926,0.000027,NaN,0.005076,1.114601e-46,0.002819,0.000038,0.023995,0.030847
2021-01-29,0.001518,0.006403,0.001306,0.006997,0.001573,0.000003,0.003703,0.001631,6.410535e-14,0.006438,...,0.012773,0.022511,0.000013,NaN,0.005100,5.573003e-47,0.004445,0.000019,0.028310,0.023182
2021-01-30,0.000967,0.007251,0.001346,0.008499,0.000786,0.083335,0.003484,0.001141,3.205268e-14,0.006157,...,0.012148,0.018314,0.000007,NaN,0.005099,2.786502e-47,0.004417,0.000010,0.025831,0.017958


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-28,0.001451,0.011517,0.002804,0.009027,0.006463,0.008891,0.005715,0.003500,0.000295,0.003870,...,0.011149,0.023742,0.000755,0.0,0.003217,0.001087,0.004618,0.000222,0.036962,0.025440
2021-01-29,0.000967,0.011357,0.002852,0.006974,0.009619,0.007319,0.005903,0.002596,0.000359,0.004113,...,0.008516,0.024872,0.000618,0.0,0.003396,0.006837,0.004070,0.000111,0.034403,0.021996
2021-01-30,0.000515,0.011139,0.002769,0.006770,0.007516,0.003659,0.005713,0.002585,0.000410,0.004110,...,0.013616,0.024021,0.001003,0.0,0.003896,0.006181,0.003530,0.000406,0.032043,0.020828


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-28,0.007280,0.003331,NaN,0.007110,0.006392,0.006559,0.003742,0.007688,0.007087,6.375049e-97,...,0.003234,0.008647,0.005084,0.011302,0.004010,0.010560,0.006787,0.007005,0.003002,0.002342
2021-01-29,0.006785,0.003383,NaN,0.006938,0.006130,0.006277,0.004069,0.006372,0.006420,3.187524e-97,...,0.005037,0.008168,0.004757,0.011509,0.004941,0.009572,0.006629,0.007319,0.003036,0.003699
2021-01-30,0.003393,0.003116,NaN,0.006889,0.006183,0.005404,0.004017,0.003186,0.007213,1.593762e-97,...,0.004076,0.007294,0.004512,0.013174,0.002471,0.009113,0.006213,0.007313,0.002886,0.001985


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-28      0.009147  0.008130  0.012055  0.005117  0.007349  0.008545   
2021-01-29      0.008355  0.008087  0.008892  0.005421  0.007505  0.002419   
2021-01-30      0.004177  0.004043  0.004446  0.002710  0.003753  0.001209   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-28      0.008522  0.007824  0.009645  0.004015  ...  0.000804   
2021-01-29      0.007617  0.007504  0.007197  0.003791  ...  0.001788   
2021-01-30      0.003808  0.003752  0.003598  0.001896  ... -0.000093   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-28      0.002147  0.002907  0.002609   0.002829  0.006000  0.006846   
2021-01-29      0.003806  0.002677  0.002076   0.004903  0.011256  0.003934   
2021-01-30      0.002809  0.001513  0.001038   0.003606  0.005628  0.002222   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-28           -1.0  0.001041  0.000285  
2021-01-29           -1.0  0.000521  0.000142  
2021-01-30           -1.0  0.000260  0.000071  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-28,0.026800,0.003883,NaN,0.013696,0.009397,0.017171,0.003558,0.006028,0.005738,NaN,...,0.013470,0.011025,0.003004,0.002065,1.295753e-09,0.010917,0.007757,0.013587,0.005739,0.005824
2021-01-29,0.028795,0.001942,NaN,0.014766,0.009611,0.016975,0.004024,0.004866,0.004264,NaN,...,0.009071,0.010445,0.012304,0.001032,6.478766e-10,0.011086,0.008828,0.012593,0.007290,0.002912
2021-01-30,0.014397,0.000971,NaN,0.010301,0.005530,0.013955,0.004336,0.002433,0.003524,NaN,...,0.010507,0.008960,0.008569,0.003423,3.239383e-10,0.011030,0.004414,0.008540,0.006775,0.001456


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-28      0.029349  0.037635  0.007219  0.007895  0.030221  0.004464   
2021-01-29      0.014674  0.016595  0.003610  0.003947  0.015110  0.002232   
2021-01-30      0.007337  0.008298  0.001805  0.001974  0.007555  0.001116   

Province_State                                        ...   Wyoming            \
Admin2          Butler   Calhoun  Chambers  Cherokee  ...      Park    Platte   
2021-01-28      0.0004  0.020056  0.015625  0.038891  ...  0.014410  0.012609   
2021-01-29      0.0002  0.012193  0.007813  0.019446  ...  0.007205  0.006304   
2021-01-30      0.0001  0.006096  0.003906  0.009723  ...  0.003603  0.003152   

Province_State                                                         \
Admin2          Sheridan      Sublette Sweetwater     Teton     Uinta   
2021-01-28      0.007002  1.030664e-08   0.008967  0.025244  0.001672   
2021-01-29      0.003501  5.153319e-09   0.004484  0.012622  0.000836   
2021-01-30      0.001750  2.576659e-09   0.002242  0.006311  0.000418   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-01-28      -0.501961  0.000369  1.092752e-07  
2021-01-29      -0.501961  0.000184  5.463759e-08  
2021-01-30      -0.501961  0.000092  2.731880e-08  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210130,AK,52470,NaN,NaN,NaN,totalTestsViral,1495885,39.0,1205.0,...,52470,0,3,7dcb430b5b094ed47dd102d1e3f18631f37b06dd,0,0,0,0,0,NaN
1,20210130,AL,455582,96227.0,1767953.0,NaN,totalTestsPeopleViral,2127308,1879.0,41859.0,...,2223535,0,376,c2749dcddcbfd6bf3c9b7ac22df91f610a801b9f,0,0,0,0,0,NaN
2,20210130,AR,294387,59482.0,2185289.0,NaN,totalTestsViral,2420194,911.0,13599.0,...,2479676,7,47,9607d122969ae43c6660b5930cb10761ebca79fc,0,0,0,0,0,NaN
3,20210130,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,bb228e9a495398b145df6eb892d379696dabf97a,0,0,0,0,0,NaN
4,20210130,AZ,753379,47715.0,2734463.0,NaN,totalTestsViral,6643932,3828.0,52006.0,...,3487842,76,435,af89eb013e093ac6706e3b4511750943b117cbac,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210130,AK,52470.0,NaN,NaN,NaN,totalTestsViral,1495885.0,39.0,1205.0,...,52470,0,3,7dcb430b5b094ed47dd102d1e3f18631f37b06dd,0,0,0,0,0,NaN
1,20210130,AL,455582.0,96227.0,1767953.0,NaN,totalTestsPeopleViral,2127308.0,1879.0,41859.0,...,2223535,0,376,c2749dcddcbfd6bf3c9b7ac22df91f610a801b9f,0,0,0,0,0,NaN
2,20210130,AR,294387.0,59482.0,2185289.0,NaN,totalTestsViral,2420194.0,911.0,13599.0,...,2479676,7,47,9607d122969ae43c6660b5930cb10761ebca79fc,0,0,0,0,0,NaN
3,20210130,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,bb228e9a495398b145df6eb892d379696dabf97a,0,0,0,0,0,NaN
4,20210130,AZ,753379.0,47715.0,2734463.0,NaN,totalTestsViral,6643932.0,3828.0,52006.0,...,3487842,76,435,af89eb013e093ac6706e3b4511750943b117cbac,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-30,AK,52470,NaN,NaN,NaN,totalTestsViral,1495885,39.0,1205.0,NaN,...,52470,0,3,7dcb430b5b094ed47dd102d1e3f18631f37b06dd,0,0,0,0,0,NaN
2021-01-30,AL,455582,96227.0,1767953.0,NaN,totalTestsPeopleViral,2127308,1879.0,41859.0,NaN,...,2223535,0,376,c2749dcddcbfd6bf3c9b7ac22df91f610a801b9f,0,0,0,0,0,NaN
2021-01-30,AR,294387,59482.0,2185289.0,NaN,totalTestsViral,2420194,911.0,13599.0,280.0,...,2479676,7,47,9607d122969ae43c6660b5930cb10761ebca79fc,0,0,0,0,0,NaN
2021-01-30,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,bb228e9a495398b145df6eb892d379696dabf97a,0,0,0,0,0,NaN
2021-01-30,AZ,753379,47715.0,2734463.0,NaN,totalTestsViral,6643932,3828.0,52006.0,984.0,...,3487842,76,435,af89eb013e093ac6706e3b4511750943b117cbac,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-30,AK,52470.0,NaN,NaN,NaN,totalTestsViral,1495885.0,39.0,1205.0,NaN,...,52470,0,3,7dcb430b5b094ed47dd102d1e3f18631f37b06dd,0,0,0,0,0,NaN
2021-01-30,AL,455582.0,96227.0,1767953.0,NaN,totalTestsPeopleViral,2127308.0,1879.0,41859.0,NaN,...,2223535,0,376,c2749dcddcbfd6bf3c9b7ac22df91f610a801b9f,0,0,0,0,0,NaN
2021-01-30,AR,294387.0,59482.0,2185289.0,NaN,totalTestsViral,2420194.0,911.0,13599.0,280.0,...,2479676,7,47,9607d122969ae43c6660b5930cb10761ebca79fc,0,0,0,0,0,NaN
2021-01-30,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,bb228e9a495398b145df6eb892d379696dabf97a,0,0,0,0,0,NaN
2021-01-30,AZ,753379.0,47715.0,2734463.0,NaN,totalTestsViral,6643932.0,3828.0,52006.0,984.0,...,3487842,76,435,af89eb013e093ac6706e3b4511750943b117cbac,0,0,0,0,0,NaN
